In [4]:
import numpy as np
from matplotlib import pyplot as plt
from astropy.nddata import Cutout2D
from astropy import units
from astropy.io import fits
from astropy.modeling.models import Gaussian2D
import glob
from astropy.visualization import astropy_mpl_style
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from clustar.core import ClustarData
from clustar.search import Clustar
from astropy.visualization import ZScaleInterval
from astropy.io import fits
import random
from scipy.ndimage import rotate


In [5]:
cs = Clustar(radius_factor = 0.75, threshold = 0.025, )
cs.run(directory='data/test_pos')

File 'Serpens_South_C7': 100%|██████████| 8/8 [00:02<00:00,  2.70file/s, num_errors=0, num_flags=5]


In [11]:
objects = (cs.data)

# print(objects[0].clean())

# object_0 = (objects[0].Params)

# print(object_0.radius_factor() )

cs.display(category='all')

for i in range(len(objects)):
    objects[i].identify()

TypeError: 'Clustar' object is not iterable

In [16]:
cs.data[0]

AttributeError: 'ClustarData' object has no attribute 'display'